In [1]:
import wandb
import os
os.environ["WANDB_PROJECT"]="QLoRA_Instruction_finetune_02"

wandb.login()

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: aeolian83. Use `wandb login --relogin` to force relogin


True

In [2]:
from datasets import load_dataset
from huggingface_hub import login
from dotenv import load_dotenv

load_dotenv()


login(token= os.environ["HF_TOKEN"])

Token has not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /home/aeolian83/.cache/huggingface/token
Login successful


In [3]:
ko_instruction_01 = load_dataset("squarelike/OpenOrca-gugugo-ko", cache_dir="/mnt/t7/.cache/huggingface/datasets")

In [4]:
ko_instruction_01 = ko_instruction_01.shuffle(seed=2160)

In [5]:
ko_instruction_01 = ko_instruction_01['train'].train_test_split(test_size=0.95)

In [6]:
ko_instruction_01['train'][1500]['question']

'질문: 아래 문맥을 고려할 때: 1953: 콜로라도의 가을과 겨울을 거치면서 메리 스펜서와 프레드 윌슨은 이상적인 삶을 살아갑니다. 메리는 프레드와 결혼하기 위해 대학을 중퇴합니다(6개월이 남았습니다). 메리의 완벽한 결혼식은 메리가 좋아하는 영화의 행복한 결말을 반영합니다. 1969: 윌슨 부부의 결혼 16주년입니다. 출근길에 프레드는 성공한 세무 컨설턴트인 프레드는 자신의 마가렛에게 자신의 옷장에서 보드카를 발견했다고 말하며 마가렛에게 아내를 지켜달라고 부탁합니다. 하지만 메리는 미용실을 향합니다. 하지만 비행기 사무실에서 메리는 지루하고 불행한 교외 생활에서 벗어나기 위해 바하마의 나소에 가는 편도 항공권을 구입합니다. 비행기에 탑승한 그녀는 지난해 결혼 1주년 파티의 악몽을 떠올리며, 프레드가 술에 취해 금발의 이혼녀와 농담을 주고받았고, 그녀는 술과 카사블랑카의 재방송을 통해 피난처를 찾았던 기억을 떠올립니다. 환승지에서 그녀는 집에 전화를 걸어 올해 결혼 1주년 파티가 완전히 다른 종류의 재앙이었다는 사실을 알게 됩니다. 그녀의 십대 딸 매지는 약물에 취한 어머니를 발견한 기억을 떠올리며 어머니의 전화를 두려워합니다. 메리는 나소로 향하는 길에 1953년 이후 한 번도 만나지 못한 대학 친구 프롤을 만납니다. 메리는 결혼 생활에 정착한 반면, 프롤은 일련의 결혼한 남성들의 정부로 살아가며 다소 자유분방하고 쾌락주의적인 삶을 살며 성 혁명을 완전히 받아들였습니다. 그녀는 새 연인 샘을 만나기 위해 나소로 향하고 있습니다. 메리는 프레드로부터 도망쳐야 한다고 말하자 프롤은 그녀를 돌봐주겠다고 약속합니다.  유효한 제목을 맞춰보세요!\n답변:'

In [7]:
ko_instruction_01['train'][1500]['system_prompt']

''

In [8]:
ko_instruction_01['train'][1500]['response']

'"나소로의 탈출: 결혼, 불륜, 우정의 이야기"'

In [9]:
import torch
import transformers
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, TrainingArguments
from peft import LoraConfig, PeftModel, get_peft_model
from trl import SFTTrainer

In [10]:
checkpoint_dir = "./checkpoint"
model_id = "beomi/llama-2-ko-7b"
device_map = {"": 0}

In [11]:
tokenizer = AutoTokenizer.from_pretrained(model_id, cache_dir="/mnt/t7/.cache/huggingface/models")

In [12]:
def format_instruction(sample):
    system_prompt = f"### System: {sample['system_prompt']}"
    input = f"### Human: {sample['question']}" if len(sample["question"]) > 0 else None
    output = f"### Assistant\n{sample['response']}"
    # join all the parts together
    prompt = "\n\n".join([i for i in [system_prompt, input, output] if i is not None])
    return prompt

# template dataset to add prompt to each sample
def template_dataset(sample):
    sample["text"] = f"{format_instruction(sample)}{tokenizer.eos_token}"
    return sample

In [13]:
train_dataset = ko_instruction_01['train'].map(template_dataset, remove_columns=list(ko_instruction_01['train'].features), num_proc=10)

Map (num_proc=10):   0%|          | 0/112014 [00:00<?, ? examples/s]

In [14]:
train_dataset

Dataset({
    features: ['text'],
    num_rows: 112014
})

In [15]:
train_dataset["text"][244]

'### System: 당신은 인공지능 비서입니다. 사용자가 과제를 부여할 것입니다. 당신의 목표는 가능한 한 충실하게 과제를 완료하는 것입니다. 과제를 수행하는 동안 단계별로 생각하고 단계를 정당화하세요.\n\n### Human: 페르시아어로 된 문장이 주어집니다. 당신의 임무는 페르시아 문장을 스페인어로 번역하는 것입니다.\nQ: بعد از گذشت دقیقا یک سال ما وسیله ای داشتیم با 14 درجه آزادی ، همه سنسورها ، تمام میکرو پروسسورها ، همه چیز در داخلش.\nA: \n\n### Assistant\n정확히 1년 후, 우리는 14개의 자유도를 가진 디바이스, 모든 센서, 모든 마이크로프로세서를 가지고 있었습니다.</s>'